# PSO implementation for hyperparameter tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch torchvision matplotlib numpy --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import json
import os
import time


In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [ ]:
learning_rate = 0.00934
hidden_size = 97
batch_size = 241
num_epochs = 15000
input_size = 28 * 28
num_classes = 10
save_dir = "/content/drive/MyDrive/PSO_Training_Results"
os.makedirs(save_dir, exist_ok=True)


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 32.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.01MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 7.85MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.60MB/s]


In [ ]:
model = SimpleNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
start_epoch = 0
train_loss_history = []
train_accuracy_history = []

ckpt_path = os.path.join(save_dir, "model_latest.pth")
if os.path.exists(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"]
    train_loss_history = checkpoint["loss_history"]
    train_accuracy_history = checkpoint["acc_history"]
    print(f"Resumed from epoch {start_epoch}")


In [ ]:
start_time = time.time()

for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    train_loss_history.append(epoch_loss)
    train_accuracy_history.append(epoch_accuracy)

    if (epoch + 1) % 100 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # 🔁 Save model checkpoint every 500 epochs
    if (epoch + 1) % 500 == 0:
        torch.save({
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss_history": train_loss_history,
            "acc_history": train_accuracy_history
        }, ckpt_path)


Epoch [1/15000] - Loss: 0.4570, Accuracy: 86.31%
Epoch [100/15000] - Loss: 0.0731, Accuracy: 97.86%
Epoch [200/15000] - Loss: 0.0642, Accuracy: 98.26%
Epoch [300/15000] - Loss: 0.0532, Accuracy: 98.72%
Epoch [400/15000] - Loss: 0.0836, Accuracy: 98.53%
Epoch [500/15000] - Loss: 0.0419, Accuracy: 99.16%
Epoch [600/15000] - Loss: 0.0460, Accuracy: 99.27%
Epoch [700/15000] - Loss: 0.0802, Accuracy: 98.95%
Epoch [800/15000] - Loss: 0.0607, Accuracy: 99.26%
Epoch [900/15000] - Loss: 0.0397, Accuracy: 99.50%
Epoch [1000/15000] - Loss: 0.0298, Accuracy: 99.59%
Epoch [1100/15000] - Loss: 0.0576, Accuracy: 99.48%
Epoch [1200/15000] - Loss: 0.0736, Accuracy: 99.41%
Epoch [1300/15000] - Loss: 0.0648, Accuracy: 99.53%
Epoch [1400/15000] - Loss: 0.0455, Accuracy: 99.61%
Epoch [1500/15000] - Loss: 0.0766, Accuracy: 99.48%
Epoch [1600/15000] - Loss: 0.0581, Accuracy: 99.60%
Epoch [1700/15000] - Loss: 0.0672, Accuracy: 99.57%
